In [554]:
#importing libraries 
import pandas as pd
from geopy import distance
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning) 
import datetime
import calendar
import re

In [555]:
import warnings
warnings.filterwarnings("ignore")


In [616]:
#getting granular level insights from date6\
def add_datepart(df, fldname, drop=True):
    
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
    attr = ['Year', 'Month', 'Week', 'Day','hour','minute','second']
    
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    
    df[targ_pre + "MonthName"] = df[targ_pre + "Month"].astype(int).apply(lambda x: calendar.month_abbr[x])
    df[targ_pre + "MonthKey"] =  df[targ_pre + "Year"].astype(str)+"_"+df[targ_pre + "Month"].astype(str)
    df[targ_pre + "WeekKey"] =  df[targ_pre + "Year"].astype(str)+"_"+df[targ_pre + "Week"].astype(str)
        
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

In [617]:
# load the dataset 
df = pd.read_excel('GJ12b.xltm')

In [888]:
df.shape

(1449, 11)

In [916]:
df.head()

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long,Distance,Time_Difference
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,2021-10-16 10:10:00,2021-10-16 10:15:00,0 days 00:05:00,0,27.99536,73.31652,13.003621,79.990784,1805.03,0 days 00:00:00
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,2021-10-16 10:15:00,2021-10-16 10:20:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784,1805.03,0 days 00:05:00
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,2021-10-16 10:20:00,2021-10-16 10:25:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784,1805.03,0 days 00:05:00
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,2021-10-16 10:25:00,2021-10-16 10:30:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784,1805.03,0 days 00:05:00
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,2021-10-16 10:30:00,2021-10-16 10:35:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784,1805.03,0 days 00:05:00


In [618]:
# Display the data set
df.tail()


,"Lat,Lngt",Period
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55
1448,"28.039583,73.430438",21/10 11:55 : 21/10 11:57
1449,NaN,16/10 10:10 : 21/10 11:57


In [921]:
# 
dt1 = '2021-10-16 10:00:00'
d1 = pd.to_datetime(dt1)
dt2 = '2021-10-21 11:25:00'
d2 = pd.to_datetime(dt2)

a = d2-d1
a



Timedelta('5 days 01:25:00')

In [620]:
#df_gj['start_dt'] = df_gj['Period'].map(lambda x:"2021"+"-"+str(x)[0:2]+"-"+str(x)[3:5]+" "+str(x)[6:11])
df['start_dt'] = df['Period'].map(lambda x:"2021"+"-"+str(x)[3:5]+"-"+str(x)[0:2]+" "+str(x)[6:11])
df['start_dt'].head(2)

0    2021-10-16 10:10
1    2021-10-16 10:15
Name: start_dt, dtype: object

In [621]:
#df_gj['end_dt'] = df_gj['Period'].map(lambda x:"2021"+"-"+str(x)[13:16]+"-"+str(x)[17:20]+" "+str(x)[20:25])
df['end_dt'] = df['Period'].map(lambda x:"2021"+"-"+str(x)[17:20]+"-"+str(x)[13:16]+" "+str(x)[20:25])
df['end_dt'].head(2)

0    2021-10 - 16 10:15
1    2021-10 - 16 10:20
Name: end_dt, dtype: object

In [622]:
df['start_dt'] = pd.to_datetime(df['start_dt'])

In [623]:
df['end_dt'] = pd.to_datetime(df['end_dt'])

In [624]:
df['Time_diff'] = df['end_dt'] - df['start_dt']

In [626]:
df['Time_diff_new'] = 0

In [627]:
for i in range(df.shape[0] - 1):
    df['Time_diff_new'][i+1] = (datetime.datetime.min +(df['end_dt'][i+1] - df['end_dt'][i])).time()

# .time() gets the time from datetime object, if you need both days and hours, simply use this line instead:
df['Time_diff_new'][i+1] = df['end_dt'][i+1] - df['end_dt'][i]

In [628]:
df['Time_diff_new'].value_counts()

00:05:00           1446
00:02:00              1
0 days 00:00:00       1
01:10:00              1
0                     1
Name: Time_diff_new, dtype: int64

In [890]:
df['Time_Difference'] = [x.iloc[-1] - x.iloc[0] for x in df['end_dt'].rolling(2) if len(x)>0]

In [894]:
df[['Time_diff','Time_Difference']]

,Time_diff,Time_Difference
0,0 days 00:05:00,0 days 00:00:00
1,0 days 00:05:00,0 days 00:05:00
2,0 days 00:05:00,0 days 00:05:00
3,0 days 00:05:00,0 days 00:05:00
4,0 days 00:05:00,0 days 00:05:00
...,...,...
1444,0 days 00:05:00,0 days 00:05:00
1445,0 days 00:05:00,0 days 00:05:00
1446,0 days 00:05:00,0 days 00:05:00
1447,0 days 00:05:00,0 days 00:05:00


In [895]:
df['Time_Difference'].value_counts()

0 days 00:05:00    1446
0 days 01:10:00       1
0 days 00:02:00       1
0 days 00:00:00       1
Name: Time_Difference, dtype: int64

In [896]:
df['Time_Difference'].sum()

Timedelta('5 days 01:42:00')

In [ ]:
# Checking rows/cols
df.shape

In [722]:
#splitting lat/long col
df[['lat', 'long']] = df['Lat,Lngt'].str.split(',', 1, expand=True)

In [724]:
df

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,2021-10-16 10:10:00,2021-10-16 10:15:00,0 days 00:05:00,0,27.99536,73.31652,13.003621,79.990784
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,2021-10-16 10:15:00,2021-10-16 10:20:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,2021-10-16 10:20:00,2021-10-16 10:25:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,2021-10-16 10:25:00,2021-10-16 10:30:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,2021-10-16 10:30:00,2021-10-16 10:35:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
...,...,...,...,...,...,...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,2021-10-21 11:35:00,2021-10-21 11:40:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,2021-10-21 11:40:00,2021-10-21 11:45:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,2021-10-21 11:45:00,2021-10-21 11:50:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,2021-10-21 11:50:00,2021-10-21 11:55:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438


In [725]:
df = df[0:1449]

In [777]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   Lat,Lngt       1449 non-null   object         
 1   Period         1449 non-null   object         
 2   start_dt       1449 non-null   datetime64[ns] 
 3   end_dt         1449 non-null   datetime64[ns] 
 4   Time_diff      1449 non-null   timedelta64[ns]
 5   Time_diff_new  1449 non-null   object         
 6   dealer_lat     1449 non-null   float64        
 7   dealer_long    1449 non-null   float64        
 8   lat            1449 non-null   float64        
 9   long           1449 non-null   float64        
 10  Distance       1449 non-null   float64        
dtypes: datetime64[ns](2), float64(5), object(3), timedelta64[ns](1)
memory usage: 124.6+ KB


In [727]:
df['dealer_lat'] = 27.99536

In [728]:
df['dealer_long'] = 73.31652

In [729]:
df

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,2021-10-16 10:10:00,2021-10-16 10:15:00,0 days 00:05:00,0,27.99536,73.31652,13.003621,79.990784
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,2021-10-16 10:15:00,2021-10-16 10:20:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,2021-10-16 10:20:00,2021-10-16 10:25:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,2021-10-16 10:25:00,2021-10-16 10:30:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,2021-10-16 10:30:00,2021-10-16 10:35:00,0 days 00:05:00,00:05:00,27.99536,73.31652,13.003621,79.990784
...,...,...,...,...,...,...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,2021-10-21 11:35:00,2021-10-21 11:40:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,2021-10-21 11:40:00,2021-10-21 11:45:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,2021-10-21 11:45:00,2021-10-21 11:50:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,2021-10-21 11:50:00,2021-10-21 11:55:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438


In [730]:
# checking data types for type conversion
df.dtypes

Lat,Lngt                  object
Period                    object
start_dt          datetime64[ns]
end_dt            datetime64[ns]
Time_diff        timedelta64[ns]
Time_diff_new             object
dealer_lat               float64
dealer_long              float64
lat                       object
long                      object
dtype: object

In [731]:
# lat / long is object type -> we must convert it to a floating point number 
df["lat"]= df["lat"].astype(float)
df["long"]= df["long"].astype(float)

In [732]:
df[0:1]

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,2021-10-16 10:10:00,2021-10-16 10:15:00,0 days 00:05:00,0,27.99536,73.31652,13.003621,79.990784


In [734]:
import numpy as np

### Calculating straight line distance using haversine method

In [735]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [736]:
dealer_lat = 27.99536
dealer_lon = 73.31652
distances_km = []
for row in df.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat, dealer_lon, row.lat, row.long)
   )

In [737]:
df['Distance'] = distances_km

In [931]:
df['Status'] = np.where(df['Distance']!= df['Distance'].min(), 'In-transit', 'Unloading-Point')


In [933]:
df.shape

(1449, 13)

In [932]:
df.iloc[np.where(df.Distance==df.Distance.min())]

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long,Distance,Time_Difference,Status
1442,"28.039583,73.430438",21/10 11:25 : 21/10 11:30,2021-10-21 11:25:00,2021-10-21 11:30:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1443,"28.039583,73.430438",21/10 11:30 : 21/10 11:35,2021-10-21 11:30:00,2021-10-21 11:35:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,2021-10-21 11:35:00,2021-10-21 11:40:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,2021-10-21 11:40:00,2021-10-21 11:45:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,2021-10-21 11:45:00,2021-10-21 11:50:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,2021-10-21 11:50:00,2021-10-21 11:55:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point
1448,"28.039583,73.430438",21/10 11:55 : 21/10 11:57,2021-10-21 11:55:00,2021-10-21 11:57:00,0 days 00:02:00,00:02:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:02:00,Unloading-Point


# removing other rows; keeping only single unloading point

In [934]:
# removing other rows; keeping only single unloading point
df1_gj = df[0:1443].copy()

In [1128]:
df1_gj.shape

(1443, 14)

In [941]:
df1_gj.head(1)

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long,Distance,Time_Difference,Status
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,2021-10-16 10:10:00,2021-10-16 10:15:00,0 days 00:05:00,0,27.99536,73.31652,13.003621,79.990784,1805.03,0 days,In-transit


In [1034]:
df1_gj.tail(1)

,"Lat,Lngt",Period,start_dt,end_dt,Time_diff,Time_diff_new,dealer_lat,dealer_long,lat,long,Distance,Time_Difference,Status,diff_Stop
1442,"28.039583,73.430438",21/10 11:25 : 21/10 11:30,2021-10-21 11:25:00,2021-10-21 11:30:00,0 days 00:05:00,00:05:00,27.99536,73.31652,28.039583,73.430438,12.22,0 days 00:05:00,Unloading-Point,NaT


In [936]:
df1_gj['Status'].value_counts()

In-transit         1442
Unloading-Point       1
Name: Status, dtype: int64

In [1031]:
df1_gj['Time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df1_gj['end_dt'].rolling(2) if len(x)>0]

In [1032]:
df1_gj['Time_diff'].value_counts()

0 days 00:05:00    1441
0 days 01:10:00       1
0 days 00:00:00       1
Name: Time_diff, dtype: int64

In [1033]:
df1_gj['Time_diff'].sum()

Timedelta('5 days 01:15:00')

In [1035]:
dt1 = '2021-10-16 10:15:00'
d1 = pd.to_datetime(dt1)
dt2 = '2021-10-21 11:30:00'
d2 = pd.to_datetime(dt2)

a = d2-d1
a

Timedelta('5 days 01:15:00')

In [1054]:
#df1_gj

# stoppage time new

In [1056]:
df1gby = df1_gj.groupby('Lat,Lngt').agg({'Time_diff': np.sum})

In [1129]:
df1gby['Time_diff'].value_counts()

0 days 00:05:00    763
0 days 00:10:00      8
0 days 00:25:00      5
0 days 00:15:00      3
0 days 00:20:00      2
0 days 00:30:00      2
0 days 00:50:00      1
0 days 00:35:00      1
0 days 03:15:00      1
0 days 16:20:00      1
0 days 01:35:00      1
0 days 03:25:00      1
0 days 01:00:00      1
0 days 01:10:00      1
0 days 03:45:00      1
0 days 06:15:00      1
0 days 01:30:00      1
0 days 04:00:00      1
0 days 08:10:00      1
Name: Time_diff, dtype: int64

In [1059]:
df1gby_res = df1gby[df1gby['Time_diff'] >= '01:00:00']

In [1060]:
df1gby_res

,Time_diff
"Lat,Lngt",
"12.99363,80.012518",0 days 01:30:00
"12.996785,79.989242",0 days 01:00:00
"12.997075,79.989683",0 days 01:35:00
"13.003621,79.990784",0 days 03:25:00
"13.003654,79.990822",0 days 16:20:00
"13.111439,77.88025",0 days 03:15:00
"13.133839,77.356141",0 days 03:45:00
"17.347675,75.839744",0 days 01:10:00
"17.465471,75.852986",0 days 08:10:00


In [1062]:
df1gby_res['Time_diff'].mean()

Timedelta('0 days 04:35:00')

In [1063]:
df1gby_res['Time_diff'].median()

Timedelta('0 days 03:25:00')

In [1064]:
df1gby_res['Time_diff'].max()

Timedelta('0 days 16:20:00')

In [ ]:
# stoppage time old 

In [977]:
A=df1_gj.groupby(['lat', 'long']).agg({'end_dt':['min','max']})

In [978]:
A.to_excel('dddf1gj.xlsx')

In [979]:
df1 = pd.read_excel("D://Sales Logistic Project\\df1_final.xltm")

In [980]:
df1

,lat,long,min,max
0,12.869451,79.646637,2021-10-17 12:30:00,2021-10-17 12:30:00
1,12.871091,79.711763,2021-10-17 12:20:00,2021-10-17 12:20:00
2,12.873888,79.747770,2021-10-17 12:15:00,2021-10-17 12:15:00
3,12.874396,79.678234,2021-10-17 12:25:00,2021-10-17 12:25:00
4,12.877281,79.612570,2021-10-17 12:35:00,2021-10-17 12:35:00
...,...,...,...,...
791,28.091904,74.024634,2021-10-21 09:35:00,2021-10-21 09:35:00
792,28.093113,73.953741,2021-10-21 09:45:00,2021-10-21 09:45:00
793,28.093616,73.876032,2021-10-21 09:55:00,2021-10-21 09:55:00
794,28.099674,73.799597,2021-10-21 10:05:00,2021-10-21 10:05:00


In [981]:
df1['max']=pd.to_datetime(df1['max'])

In [982]:
df1['min']=pd.to_datetime(df1['min'])

In [983]:
df1['Diff'] = df1['max'] - df1['min']

In [984]:
df1['Diff'].value_counts()

0 days 00:00:00    764
0 days 00:05:00      8
0 days 00:20:00      5
0 days 00:10:00      3
0 days 00:15:00      2
0 days 00:25:00      2
0 days 01:30:00      1
0 days 00:30:00      1
0 days 03:10:00      1
0 days 16:15:00      1
0 days 08:05:00      1
0 days 00:55:00      1
0 days 03:25:00      1
0 days 03:40:00      1
0 days 06:10:00      1
0 days 01:25:00      1
0 days 03:55:00      1
0 days 00:45:00      1
Name: Diff, dtype: int64

In [985]:
df1_res = df1[df1['Diff'] >= '1:00:00']

In [990]:
df1_res['Diff'].value_counts().count()

9

In [987]:
df1_res_lessthan1hr = df1[df1['Diff'] < '1:00:00']

In [989]:
df1_res_lessthan1hr

,lat,long,min,max,Diff
0,12.869451,79.646637,2021-10-17 12:30:00,2021-10-17 12:30:00,0 days
1,12.871091,79.711763,2021-10-17 12:20:00,2021-10-17 12:20:00,0 days
2,12.873888,79.747770,2021-10-17 12:15:00,2021-10-17 12:15:00,0 days
3,12.874396,79.678234,2021-10-17 12:25:00,2021-10-17 12:25:00,0 days
4,12.877281,79.612570,2021-10-17 12:35:00,2021-10-17 12:35:00,0 days
...,...,...,...,...,...
791,28.091904,74.024634,2021-10-21 09:35:00,2021-10-21 09:35:00,0 days
792,28.093113,73.953741,2021-10-21 09:45:00,2021-10-21 09:45:00,0 days
793,28.093616,73.876032,2021-10-21 09:55:00,2021-10-21 09:55:00,0 days
794,28.099674,73.799597,2021-10-21 10:05:00,2021-10-21 10:05:00,0 days


In [904]:
787 + 9

796

In [991]:
df1_res['Diff'].mean()


Timedelta('0 days 05:17:13.333333333')

In [992]:
df1_res['Diff'].median()

Timedelta('0 days 03:40:00')

In [771]:
df1_res['Diff'].max()

Timedelta('0 days 16:15:00')

In [772]:
df1_res['Diff'].min()

Timedelta('0 days 01:25:00')

In [774]:
df1_res['Diff'].sum()

Timedelta('1 days 23:35:00')

## Dataset 2 

In [1066]:
#Read the dataset
df2 = pd.read_excel("NL01AA_1.xltm")

In [1067]:
# Checking data types -> lat / long has to be a floating point number
#df2.info()

In [1068]:
#Exploring dataset contents
df2.head()
print(df2.shape)

(529, 5)


In [1069]:
#splitting lat/long col
#df2[['lat', 'long']] = df2['LatLong'].str.split(',',1,expand=True)

In [1070]:
# Setting dealer's location 
df2['dealer_lat2'] = 13.070983
df2['dealer_long2'] = 77.452922

# Stoppage time

In [1071]:
#Stoppage time
#df2

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2
0,0,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:03:00,13.070983,77.452922
1,1,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:08:00,13.070983,77.452922
2,2,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:18:00,13.070983,77.452922
3,3,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:23:00,13.070983,77.452922
4,4,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:28:00,13.070983,77.452922
...,...,...,...,...,...,...,...
524,524,"13.081451,77.450764",13.081451,77.450764,2021-10-05 23:38:00,13.070983,77.452922
525,525,"13.081451,77.450764",13.081451,77.450764,2021-10-05 23:43:00,13.070983,77.452922
526,526,"13.081451,77.450764",13.081451,77.450764,2021-10-05 23:48:00,13.070983,77.452922
527,527,"13.081451,77.450764",13.081451,77.450764,2021-10-05 23:53:00,13.070983,77.452922


In [1072]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
# old stoppage time

In [779]:
#b=df2.groupby(['Lat', 'Long']).agg({'Date':['min','max']})

In [781]:
#b.to_excel('b.xlsx')

In [782]:
#df2_Nl0AA = pd.read_excel("D:Sales Logistic Project\\df2grpby.xltm")

In [783]:
#df2_Nl0AA['max']=pd.to_datetime(df2_Nl0AA['max'])

In [784]:
#df2_Nl0AA['min']=pd.to_datetime(df2_Nl0AA['min'])

In [785]:
#df2_Nl0AA['Diff']=df2_Nl0AA['max']-df2_Nl0AA['min']

In [1073]:
#df2_Nl0AA['Diff'].count()

In [787]:
#df2res = df2_Nl0AA[df2_Nl0AA['Diff'] >= '1:00:00']

In [1075]:
#df2res['Diff'].count()

In [1074]:
#print("Mean = ",df2res['Diff'].mean())
#print("Median=",df2res['Diff'].median())
#print("Max = ",df2res['Diff'].max())
#print("Min = ",df2res['Diff'].min())
#print("Sum = ",df2res['Diff'].sum())

In [801]:
# Time difference consistancy
#df2['Time_diff'] = 0

In [1076]:
#for i in range(df2.shape[0] - 1):
    #df2['Time_diff'][i+1] = (datetime.datetime.min +(df2['Date'][i+1] - df2['Date'][i])).time()

# .time() gets the time from datetime object, if you need both days and hours, simply use this line instead:
#df2['Time_diff'][i+1] = df2['Date'][i+1] - df2['Date'][i]

In [695]:
#df2.to_excel("nlaa.xlsx")

In [696]:
# converting lat/long to floating type

### Straight Line Distance

In [1077]:
dealer_lat2 = 13.070983
dealer_lon2 = 77.452922
distances_km = []
for row in df2.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat2, dealer_lon2, row.Lat, row.Long)
   )

In [1078]:
df2['Distance'] = distances_km

In [1079]:
df2['Status'] = np.where(df2['Distance']!= df2['Distance'].min(), 'In-transit', 'Unloading-Point')

In [1080]:
df2.iloc[np.where(df2['Distance']==df2['Distance'].min())]

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance,Status
512,512,"13.066884,77.451716",13.066884,77.451716,2021-10-05 22:32:00,13.070983,77.452922,0.47,Unloading-Point


In [1081]:
df2.shape

(529, 9)

## Sliced DataFrame 2 - Unloading Point

In [1082]:
df2_new = df2[0:513]

In [1130]:
df2_new.shape

(513, 10)

In [1083]:
df2_new.Status.value_counts()

In-transit         512
Unloading-Point      1
Name: Status, dtype: int64

In [1084]:
df2_new.tail(1)

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance,Status
512,512,"13.066884,77.451716",13.066884,77.451716,2021-10-05 22:32:00,13.070983,77.452922,0.47,Unloading-Point


In [1085]:
df2_new.head(1)

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance,Status
0,0,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:03:00,13.070983,77.452922,279.01,In-transit


In [1094]:
df2_new['Time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df2_new['Date'].rolling(2) if len(x)>0]


In [1095]:
df2_new[['Time_diff']].sum()

Time_diff   1 days 22:29:00
dtype: timedelta64[ns]

In [1096]:
df2_new['Time_diff'].value_counts()

0 days 00:05:00    443
0 days 00:10:00     28
0 days 00:06:00     25
0 days 00:09:00      8
0 days 00:07:00      5
0 days 00:08:00      2
0 days 00:21:00      1
0 days 00:00:00      1
Name: Time_diff, dtype: int64

In [1047]:
dt1 = '2021-10-04 00:03:00'
d1 = pd.to_datetime(dt1)
dt2 = '2021-10-05 22:32:00'
d2 = pd.to_datetime(dt2)

a = d2-d1
a

Timedelta('1 days 22:29:00')

### Stoppage Time

In [1097]:
df2_new_gby = df2_new.groupby('LatLong').agg({'Time_diff': np.sum})

In [1140]:
df2_new_gby.Time_diff.value_counts()

0 days 00:05:00    92
0 days 00:06:00    18
0 days 00:07:00     3
0 days 01:35:00     2
0 days 02:26:00     1
0 days 00:15:00     1
0 days 13:40:00     1
0 days 01:43:00     1
0 days 08:25:00     1
0 days 01:03:00     1
0 days 00:44:00     1
0 days 00:43:00     1
0 days 01:00:00     1
0 days 00:59:00     1
0 days 00:22:00     1
0 days 02:10:00     1
Name: Time_diff, dtype: int64

In [1133]:
df2_new_gby.to_excel('df2_new_gby.xlsx')

In [1104]:
df2_new_gby_res = df2_new_gby[df2_new_gby['Time_diff'] >='01:00:00']


In [1105]:
df2_new_gby_res

,Time_diff
LatLong,
"12.623982,78.080702",0 days 02:10:00
"12.779421,77.763307",0 days 01:03:00
"12.886374,79.596062",0 days 08:25:00
"12.994271,80.012587",0 days 01:43:00
"13.024806,80.028267",0 days 01:35:00
"13.024907,80.028187",0 days 01:00:00
"13.024988,80.028222",0 days 01:35:00
"13.024992,80.028302",0 days 02:26:00
"13.024994,80.028231",0 days 13:40:00


## Dataset 3

In [403]:
df3 = pd.read_excel("NL01AB.xltm")

In [404]:
df3

,Latitude,Longitude,Packet Time
0,13.029557,80.032257,02-Sep-2021 01:05
1,13.029557,80.032257,02-Sep-2021 01:10
2,13.027725,80.029144,02-Sep-2021 01:15
3,13.013268,80.001274,02-Sep-2021 01:20
4,13.002186,80.000305,02-Sep-2021 01:25
...,...,...,...
452,15.841319,74.541664,03-Sep-2021 15:46
453,15.841319,74.541664,03-Sep-2021 15:51
454,15.841319,74.541664,03-Sep-2021 15:56
455,15.841319,74.541664,03-Sep-2021 15:59


In [405]:
df3['Dealer_lat3'] = 15.818516
df3['Dealer_long3'] = 74.494901

In [406]:
df3 = df3[0:455]

In [407]:
df3

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3
0,13.029557,80.032257,02-Sep-2021 01:05,15.818516,74.494901
1,13.029557,80.032257,02-Sep-2021 01:10,15.818516,74.494901
2,13.027725,80.029144,02-Sep-2021 01:15,15.818516,74.494901
3,13.013268,80.001274,02-Sep-2021 01:20,15.818516,74.494901
4,13.002186,80.000305,02-Sep-2021 01:25,15.818516,74.494901
...,...,...,...,...,...
450,15.841319,74.541664,03-Sep-2021 15:36,15.818516,74.494901
451,15.841319,74.541664,03-Sep-2021 15:41,15.818516,74.494901
452,15.841319,74.541664,03-Sep-2021 15:46,15.818516,74.494901
453,15.841319,74.541664,03-Sep-2021 15:51,15.818516,74.494901


In [408]:
df3.dtypes

Latitude        float64
Longitude       float64
Packet Time      object
Dealer_lat3     float64
Dealer_long3    float64
dtype: object

In [409]:
dealer_lat3 = 15.818516
dealer_lon3 = 74.494901
distances_km = []
for row in df3.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat3, dealer_lon3, row.Latitude, row.Longitude)
   )

In [410]:
df3['Distance'] = distances_km

In [44]:
df3.to_excel("NL01AB.xlsx")

In [818]:
df3


,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance
0,13.029557,80.032257,02-Sep-2021 01:05,15.818516,74.494901,672.06
1,13.029557,80.032257,02-Sep-2021 01:10,15.818516,74.494901,672.06
2,13.027725,80.029144,02-Sep-2021 01:15,15.818516,74.494901,671.86
3,13.013268,80.001274,02-Sep-2021 01:20,15.818516,74.494901,669.96
4,13.002186,80.000305,02-Sep-2021 01:25,15.818516,74.494901,670.46
...,...,...,...,...,...,...
450,15.841319,74.541664,03-Sep-2021 15:36,15.818516,74.494901,5.61
451,15.841319,74.541664,03-Sep-2021 15:41,15.818516,74.494901,5.61
452,15.841319,74.541664,03-Sep-2021 15:46,15.818516,74.494901,5.61
453,15.841319,74.541664,03-Sep-2021 15:51,15.818516,74.494901,5.61


In [1106]:
df3['Status'] = np.where(df3['Distance']!= df3['Distance'].min(), 'In-transit', 'Unloading-Point')

In [1107]:
df3['Packet Time'] = pd.to_datetime(df3['Packet Time'])

In [1108]:
df3['Time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df3['Packet Time'].rolling(2) if len(x)>0]

In [1109]:
df3.Time_diff.value_counts()

0 days 00:05:00    393
0 days 00:06:00     61
0 days 00:00:00      1
Name: Time_diff, dtype: int64

In [1110]:
df3.Time_diff.sum()

Timedelta('1 days 14:51:00')

In [821]:
# old stoppage time

In [838]:
df3_nlab = df3.groupby(['Latitude', 'Longitude']).agg({'Packet Time':['min','max']})

In [840]:
df3_nlab.to_excel("df3_nlab.xlsx")

In [841]:
df3_nlab = pd.read_excel("D:Sales Logistic Project\\df3nlab.xltm")

In [843]:
df3_nlab['max'] = pd.to_datetime(df3_nlab['max'])

In [844]:
df3_nlab['min'] = pd.to_datetime(df3_nlab['min'])

In [845]:
df3_nlab['Diff'] = df3_nlab['max'] - df3_nlab['min']

In [913]:
df3_nlab_stop = df3_nlab[df3_nlab['Diff'] >= '1:00:00']

In [914]:
df3_nlab_stop['Diff'].count()

3

In [915]:
df3_nlab['Diff'].count()

289

In [856]:
print("Mean ",df3_nlab_stop['Diff'].mean())
print("Median ",df3_nlab_stop['Diff'].median())
print("Min ",df3_nlab_stop['Diff'].min())
print("Max ",df3_nlab_stop['Diff'].max())
print("Sum ",df3_nlab_stop['Diff'].sum())

Mean  0 days 03:20:40
Median  0 days 03:09:00
Min  0 days 01:32:00
Max  0 days 05:21:00
Sum  0 days 10:02:00


In [1011]:
df3.shape

(455, 8)

In [1010]:
df3.iloc[np.where(df3.Distance==df3.Distance.min())]

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance,Status,Time_diff
390,15.841252,74.541702,2021-09-03 10:30:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
391,15.841319,74.541664,2021-09-03 10:35:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
392,15.841319,74.541664,2021-09-03 10:40:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
393,15.841319,74.541664,2021-09-03 10:45:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
394,15.841319,74.541664,2021-09-03 10:51:00,15.818516,74.494901,5.61,Completed,0 days 00:06:00
...,...,...,...,...,...,...,...,...
450,15.841319,74.541664,2021-09-03 15:36:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
451,15.841319,74.541664,2021-09-03 15:41:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
452,15.841319,74.541664,2021-09-03 15:46:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00
453,15.841319,74.541664,2021-09-03 15:51:00,15.818516,74.494901,5.61,Completed,0 days 00:05:00


## Sliced df3 - Unloading Point

In [1113]:
df3_new = df3[0:391]

In [1134]:
df3_new.shape


(391, 9)

In [1115]:
df3_new['Time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df3_new['Packet Time'].rolling(2) if len(x)>0]

In [1116]:
df3_new['Time_diff'].value_counts()

0 days 00:05:00    335
0 days 00:06:00     55
0 days 00:00:00      1
Name: Time_diff, dtype: int64

In [1117]:
df3_new['Time_diff'].sum()

Timedelta('1 days 09:25:00')

In [1118]:
df3_new.head(1)

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance,Status,Time_diff
0,13.029557,80.032257,2021-09-02 01:05:00,15.818516,74.494901,672.06,In-transit,0 days


In [1119]:
df3_new.tail(1)

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance,Status,Time_diff
390,15.841252,74.541702,2021-09-03 10:30:00,15.818516,74.494901,5.61,Unloading-Point,0 days 00:05:00


In [1120]:
dt1 = '2021-09-02 01:05:00'
d1 = pd.to_datetime(dt1)
dt2 = '2021-09-03 10:30:00'
d2 = pd.to_datetime(dt2)

a = d2-d1
a

Timedelta('1 days 09:25:00')

In [ ]:
## Stoppage time new

In [1121]:
df3_new['Lat,Lngt'] = df3_new['Latitude'].astype(str) + df3_new['Longitude'].astype(str)

In [1123]:
df3_new_gby = df3_new.groupby('Lat,Lngt').agg({'Time_diff': np.sum})

In [1142]:
df3_new_gby.value_counts()

Time_diff      
0 days 00:05:00    235
0 days 00:06:00     39
0 days 00:20:00      2
0 days 00:11:00      2
0 days 00:10:00      2
0 days 03:15:00      1
0 days 01:37:00      1
0 days 01:02:00      1
0 days 00:56:00      1
0 days 00:52:00      1
0 days 00:21:00      1
0 days 00:16:00      1
0 days 00:15:00      1
dtype: int64

In [1125]:
df3_new_gby['Time_diff'].sum()

Timedelta('1 days 09:25:00')

In [1126]:
df3_new_gby_res = df3_new_gby[df3_new_gby['Time_diff'] >= '01:00:00']

In [1127]:
df3_new_gby_res

,Time_diff
"Lat,Lngt",
12.9955980.013062,0 days 01:37:00
13.03036380.030731,0 days 03:15:00
13.2169877.252518,0 days 01:02:00


## Calculating Distance Using Map My India API 

In [46]:
## dataset 1 
lat1=79.990784
long1=13.003621
#destination
long2=73.31652
lat2=27.99536

In [47]:
url = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/driving/79.990784,13.003621;73.31652,27.99536;17ZUL7?rtype=0&region=ind"""
r = requests.get(url)
result= json.loads(r.content)
result

{'Server': 'Adv-5400',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 2503857, 2201925.2]],
  'code': 'Ok',
  'durations': [[0, 141073.1, 120174.6]]},
 'responseCode': 200}

In [120]:
url = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.990784,13.003621;73.31652,27.99536;17ZUL7?rtype=1&region=ind"""
r = requests.get(url)
result= json.loads(r.content)
result

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 2503300.4, 2176142.4]],
  'code': 'Ok',
  'durations': [[0, 176809.3, 150276.7]]},
 'responseCode': 200}

In [119]:
2503300.4/1000

2503.3004

In [121]:
#df	 lat/long
#13.003621,79.990784
#27.99536,73.31652

# long/lat
79.990784,13.003621
73.31652,27.99536

,"Lat,Lngt",Period,lat,long,dealer_lat,dealer_long,Distance
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784,27.99536,73.31652,1805.03
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,13.003621,79.990784,27.99536,73.31652,1805.03
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,13.003621,79.990784,27.99536,73.31652,1805.03
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,13.003621,79.990784,27.99536,73.31652,1805.03
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,13.003621,79.990784,27.99536,73.31652,1805.03
...,...,...,...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,28.039583,73.430438,27.99536,73.31652,12.22
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438,27.99536,73.31652,12.22
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438,27.99536,73.31652,12.22
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438,27.99536,73.31652,12.22


In [124]:
url = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.990784,13.003621;73.31652,27.99536;17ZUL7?rtype=0&region=ind"""
r2 = requests.get(url)
result2= json.loads(r2.content)
result2

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 2503300.4, 2176142.4]],
  'code': 'Ok',
  'durations': [[0, 176809.3, 150276.7]]},
 'responseCode': 200}

In [51]:
## dataset 2 

In [52]:
# Origin
lat1 = 13.024994
long1 = 80.028231


# destination
lat2 = 13.070983
long2 = 77.452922




In [53]:
url_2 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/driving/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r2 = requests.get(url_2)
result2= json.loads(r2.content)
result2

{'Server': 'Adv-5200',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 313253.5, 2006644]],
  'code': 'Ok',
  'durations': [[0, 32885.8, 214315.4]]},
 'responseCode': 200}

In [54]:
url_2= f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/driving/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=0&region=ind"""
r2 = requests.get(url_2)
result2= json.loads(r2.content)
result2

{'Server': 'Adv-5400',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 347559.2, 2198156.2]],
  'code': 'Ok',
  'durations': [[0, 20354.9, 119960.2]]},
 'responseCode': 200}

In [55]:
#df2 

In [56]:
13.024994,80.028231
13.070983,77.452922

(13.070983, 77.452922)

In [57]:
url_2= f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/driving/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r2 = requests.get(url_2)
result2= json.loads(r2.content)
result2

{'Server': 'Adv-5200',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 313253.5, 2006644]],
  'code': 'Ok',
  'durations': [[0, 32885.8, 214315.4]]},
 'responseCode': 200}

In [58]:
347559.2/1000

347.55920000000003

In [59]:
313253.5/1000

313.2535

In [60]:
## dataset 3


In [61]:
# Origin
lat3 = 13.029557
long3 = 80.032257


# destination
lat3 = 15.818516
long3 = 74.494901

In [62]:
url_3 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/driving/80.032257,13.029557;74.494901,15.818516;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_3)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5200',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 772717.2, 2005507.6]],
  'code': 'Ok',
  'durations': [[0, 85809.2, 214134.8]]},
 'responseCode': 200}

In [65]:
835135/1000

835.135

In [74]:
100004.4/1000

100.00439999999999

In [66]:
#lowest distance api calculation

In [220]:
df2[0:1]

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance
0,0,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:03:00,13.070983,77.452922,279.01


In [221]:
df2.iloc[np.where(df2['Distance']==df2['Distance'].min())]

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance
512,512,"13.066884,77.451716",13.066884,77.451716,2021-10-05 22:32:00,13.070983,77.452922,0.47


In [222]:
url2 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url2)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 347559.2, 2172373.6]],
  'code': 'Ok',
  'durations': [[0, 24510.2, 150013.6]]},
 'responseCode': 200}

In [89]:
347559.2/1000

347.55920000000003

In [134]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [366]:
from haversine import haversine
haversine((13.024994,80.028231),(13.070983,77.452922), unit='km')

279.01386852266046

In [ ]:
13.024994,80.028231 , 13.070983,77.452922

In [116]:
url4 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=0&region=ind"""
r3 = requests.get(url4)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 347559.2, 2172373.6]],
  'code': 'Ok',
  'durations': [[0, 24510.2, 150013.6]]},
 'responseCode': 200}

In [128]:
df3[0:1]

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance
0,13.029557,80.032257,02-Sep-2021 01:05,15.818516,74.494901,672.06


In [99]:
df3.iloc[np.where(df3['Distance']==df3['Distance'].min())]

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance
390,15.841252,74.541702,03-Sep-2021 10:30,15.818516,74.494901,5.61
391,15.841319,74.541664,03-Sep-2021 10:35,15.818516,74.494901,5.61
392,15.841319,74.541664,03-Sep-2021 10:40,15.818516,74.494901,5.61
393,15.841319,74.541664,03-Sep-2021 10:45,15.818516,74.494901,5.61
394,15.841319,74.541664,03-Sep-2021 10:51,15.818516,74.494901,5.61
...,...,...,...,...,...,...
450,15.841319,74.541664,03-Sep-2021 15:36,15.818516,74.494901,5.61
451,15.841319,74.541664,03-Sep-2021 15:41,15.818516,74.494901,5.61
452,15.841319,74.541664,03-Sep-2021 15:46,15.818516,74.494901,5.61
453,15.841319,74.541664,03-Sep-2021 15:51,15.818516,74.494901,5.61


In [101]:
url4 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.032257,13.029557;74.494901,15.818516;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url4)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 835135.1, 2171237]],
  'code': 'Ok',
  'durations': [[0, 59475.8, 149792.5]]},
 'responseCode': 200}

In [102]:
835135.1/1000

835.1351

In [113]:
url5 = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.032257,13.029557;74.494901,15.818516;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url5)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 835135.1, 2171237]],
  'code': 'Ok',
  'durations': [[0, 59475.8, 149792.5]]},
 'responseCode': 200}

In [108]:
from haversine import haversine
haversine((13.029557,80.032257),(15.818516,74.494901), unit='km')



672.062209304293

In [109]:
835135/1000

835.135

# straight line 

In [411]:
# dt1
print(haversine((28.039583,73.430438),(27.99536,73.31652), unit='km'))
# dt2
print(haversine((13.066884,77.451716),(13.070983,77.452922), unit='km'))
# dt3
print(haversine((15.841252,74.541702),(15.818516,74.494901), unit='km'))





12.216011193065361
0.4741380808283665
5.608771108545808


In [412]:
df.Distance.min()

12.22

In [298]:
#df.to_excel("GJ12BV2880.xlsx")


In [299]:
#df2.to_excel("NL01AA7753.xlsx")

In [300]:
#df3.to_excel("NL01AB9978.xlsx")

In [288]:
#df['Status'] = np.where(df['Distance']!= df['Distance'].min(), 'In-transit', 'Completed')



In [291]:
#df3['Status'] = np.where(df3['Distance']!= df3['Distance'].min(), 'In-transit', 'Completed')

In [1048]:
#df3.iloc[np.where(df3['Distance']==df3['Distance'].min())]

In [1050]:
#df2[0:1]


In [1049]:
#df2.iloc[np.where(df2['Distance']==df2['Distance'].min())]

In [1051]:
#df3[0:1]


In [1052]:
#df3.iloc[np.where(df3['Distance']==df3['Distance'].min())]

# map my india api 

In [301]:
df[0:1]
# origin lat/long
13.003621,79.990784
#/origin long / lat
79.990784,13.003621

# dealer
27.99536,73.31652
73.31652,27.99536


,"Lat,Lngt",Period,lat,long,dealer_lat,dealer_long,Distance,Status
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784,27.99536,73.31652,1805.03,In-transit


In [302]:
df.iloc[np.where(df['Distance']==df['Distance'].min())]

,"Lat,Lngt",Period,lat,long,dealer_lat,dealer_long,Distance,Status
1442,"28.039583,73.430438",21/10 11:25 : 21/10 11:30,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1443,"28.039583,73.430438",21/10 11:30 : 21/10 11:35,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438,27.99536,73.31652,12.22,Completed
1448,"28.039583,73.430438",21/10 11:55 : 21/10 11:57,28.039583,73.430438,27.99536,73.31652,12.22,Completed


In [130]:
#df1 

url_one = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.990784,13.003621;73.31652,27.99536;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_one)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 2503300.4, 2176142.4]],
  'code': 'Ok',
  'durations': [[0, 176809.3, 150276.7]]},
 'responseCode': 200}

In [252]:
#df2
url_two = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028231,13.024994;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
#url_two = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.451716,13.066884;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_two)
result3= json.loads(r3.content)
result3



{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 347559.2, 2172373.6]],
  'code': 'Ok',
  'durations': [[0, 24510.2, 150013.6]]},
 'responseCode': 200}

In [132]:
#df3
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.032257,13.029557;74.494901,15.818516;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 835135.1, 2171237]],
  'code': 'Ok',
  'durations': [[0, 59475.8, 149792.5]]},
 'responseCode': 200}

In [886]:
#df_filtred

In [312]:
#df_filtred.to_excel("df_filtered.xlsx")

In [378]:
# 130 - 140
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028231,13.024994;80.028187,13.024907;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 0.5, 2172373.6]],
  'code': 'Ok',
  'durations': [[0, 0.3, 150013.6]]},
 'responseCode': 200}

In [379]:
#df3 #140
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028187,13.024907;80.028222,13.024988;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3




{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 0.1, 2172373]],
  'code': 'Ok',
  'durations': [[0, 0.1, 150013.3]]},
 'responseCode': 200}

In [318]:
347559.2 - 347558.7

0.5

In [380]:
# 160 -170

13.024988,80.028222

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028222,13.024988;80.028267,13.024806;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 16.5, 2172372.8]],
  'code': 'Ok',
  'durations': [[0, 11.9, 150013.2]]},
 'responseCode': 200}

In [335]:
#170
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028267,13.024806;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3


#13.024806,80.028267
#13.024806,80.028267

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 347575.1, 2172389.2]],
  'code': 'Ok',
  'durations': [[0, 24521.7, 150025.1]]},
 'responseCode': 200}

In [336]:
347575.1 - 347558.6

16.5

In [381]:
# 180 - 190


url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028267,13.024806;80.028302,13.024992;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 8.8, 2172389.2]],
  'code': 'Ok',
  'durations': [[0, 6.3, 150025.1]]},
 'responseCode': 200}

In [383]:
# 210 - 220 
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028302,13.024992;80.013813,12.993982;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3



{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 7948.1, 2172380.4]],
  'code': 'Ok',
  'durations': [[0, 1111.4, 150018.8]]},
 'responseCode': 200}

In [385]:
# 220 -230 

12.994271,80.012587


url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.013813,12.993982;80.012587,12.994271;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 128.8, 2178912]],
  'code': 'Ok',
  'durations': [[0, 72.2, 150690.5]]},
 'responseCode': 200}

In [386]:
# 240 - 250
# 12.994271,80.012587
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.012587,12.994271;80.004533,13.000769;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 2079.8, 2179040.8]],
  'code': 'Ok',
  'durations': [[0, 450, 150771.5]]},
 'responseCode': 200}

In [387]:
# 250 
#13.000769,80.004533

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.004533,13.000769;79.757084,12.874628;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 32055.8, 2176961]],
  'code': 'Ok',
  'durations': [[0, 2443.7, 150321.5]]},
 'responseCode': 200}

In [389]:
# 260 - 270 
12.874628,79.757084

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.757084,12.874628;79.596062,12.886374;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 18111.2, 2213859]],
  'code': 'Ok',
  'durations': [[0, 1305.8, 152860]]},
 'responseCode': 200}

In [390]:
18111.2/1000

18.1112

In [346]:
# 270 
#12.886374,79.596062
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.596062,12.886374;77.452922,13.070983;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 293941.2, 2226836.8]],
  'code': 'Ok',
  'durations': [[0, 20508, 153788.6]]},
 'responseCode': 200}

In [391]:
# 360 - 370 

#12.886389,79.59584

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.596062,12.886374;79.59584,12.886389;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 13.5, 2226836.8]],
  'code': 'Ok',
  'durations': [[0, 1, 153788.6]]},
 'responseCode': 200}

In [392]:
# 370 - 380 
# 12.912777,79.375644
79.375644,12.912777

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.59584,12.886389;79.375644,12.912777;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 24912.8, 2226823.2]],
  'code': 'Ok',
  'durations': [[0, 1792.5, 153787.6]]},
 'responseCode': 200}

In [270]:
# 380-390

12.912777,79.375644
12.914707,79.06848


url_th = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.375644,12.912777;79.06848,12.914707;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_th)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202110.19.5222',
 'results': {'distances': [[0, 36312.3, 2208661]],
  'code': 'Ok',
  'durations': [[0, 2623.4, 149994]]},
 'responseCode': 200}

In [394]:
# 390 -400

#12.817623,78.750444


url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/79.06848,12.914707;78.750444,12.817623;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 37530.5, 2177447]],
  'code': 'Ok',
  'durations': [[0, 2434.1, 153353.6]]},
 'responseCode': 200}

In [273]:
import requests
import json
# 400 - 410 
#12.610238,78.553413
long1 =78.750444
lat1 = 12.817623
lat2 = 78.553413
long2 = 12.610238
U_str = "https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/" + str(long1) + "," + str(lat1) + ";" + str(long2) + "," + str(lat2) +";17ZUL7?rtype=1&region=ind"
#url_three = f"https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/{long1},{lat1};{long2},{lat2};17ZUL7?rtype=1&region=ind"
#r3 = requests.get(url_three)
r3 = requests.get(U_str)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202110.19.5222',
 'results': {'distances': [[0, None, 2256357.2]],
  'code': 'Ok',
  'durations': [[0, None, 150764.8]]},
 'responseCode': 200}

In [396]:
# 410 - 420 

12.548514,78.43192

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/78.553413,12.610238;78.43192,12.548514;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 15296.6, 2253167]],
  'code': 'Ok',
  'durations': [[0, 978.8, 152712.3]]},
 'responseCode': 200}

In [397]:
# 420 -  430 
# 12.549217,78.196462

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/78.43192,12.548514;78.196462,12.549217;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 31609, 2237870.4]],
  'code': 'Ok',
  'durations': [[0, 2156.1, 151733.5]]},
 'responseCode': 200}

In [398]:
# 430 - 440 
12.623982,78.080702


url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/78.196462,12.549217;78.080702,12.623982;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 15746.2, 2206261.6]],
  'code': 'Ok',
  'durations': [[0, 1122.1, 149577.4]]},
 'responseCode': 200}

In [399]:
# 450 -  460
12.6453,78.048782
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/78.080702,12.623982;78.048782,12.6453;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 4391.8, 2190515.2]],
  'code': 'Ok',
  'durations': [[0, 316.7, 148455.3]]},
 'responseCode': 200}

In [400]:
#  460 - 470 
12.76176,77.794898

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/78.080702,12.623982;77.794898,12.76176;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 35798.5, 2190515.2]],
  'code': 'Ok',
  'durations': [[0, 2640.7, 148455.3]]},
 'responseCode': 200}

In [401]:
# 470 - 480 
12.779421,77.763307

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.794898,12.76176;77.763307,12.779421;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 4203.4, 2154716.8]],
  'code': 'Ok',
  'durations': [[0, 518.1, 145814.6]]},
 'responseCode': 200}

In [402]:
# 480 - 490 
12.862788,77.647369

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.763307,12.779421;77.647369,12.862788;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3


{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 16701.6, 2154583.8]],
  'code': 'Ok',
  'durations': [[0, 1399.4, 146006.3]]},
 'responseCode': 200}

In [403]:
#490 -500
77.647369,12.862788
77.475084,13.041768
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.647369,12.862788;77.475084,13.041768;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3


{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 37502.7, 2146266.2]],
  'code': 'Ok',
  'durations': [[0, 2254.8, 145418.4]]},
 'responseCode': 200}

In [404]:
# 500 - 510
#13.056666,77.473271

url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.475084,13.041768;77.473271,13.056666;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 1910.7, 2131702.8]],
  'code': 'Ok',
  'durations': [[0, 126.5, 143410.7]]},
 'responseCode': 200}

In [405]:
# 520 
#13.081451,77.450764


url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/77.473271,13.056666;77.450764,13.081451;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'Adv-5800',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 5309.4, 2131777.6]],
  'code': 'Ok',
  'durations': [[0, 874.7, 143549.2]]},
 'responseCode': 200}

In [406]:
df2.Distance.min()

0.47

In [412]:
df2.iloc[np.where(df2.Distance==df2.Distance.min())]

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance,Status
512,512,"13.066884,77.451716",13.066884,77.451716,2021-10-05 22:32:00,13.070983,77.452922,0.47,Completed


In [415]:
df2[0:1]

,Unnamed: 0,LatLong,Lat,Long,Date,dealer_lat2,dealer_long2,Distance,Status
0,0,"13.024994,80.028231",13.024994,80.028231,2021-10-04 00:03:00,13.070983,77.452922,279.01,In-transit


In [416]:
url_three = f"""https://apis.mapmyindia.com/advancedmaps/v1/95846a48eeef5b6c17863d2367c75349/distance_matrix/trucking/80.028231,13.024994;77.451716,13.066884;17ZUL7?rtype=1&region=ind"""
r3 = requests.get(url_three)
result3= json.loads(r3.content)
result3

{'Server': 'ETA-5111',
 'version': '202109.19.5222',
 'results': {'distances': [[0, 348225, 2175588.2]],
  'code': 'Ok',
  'durations': [[0, 30624.3, 177989.5]]},
 'responseCode': 200}

In [50]:
## 15-November-2021
import numpy as np

### check data consistency in time interval
### identify the unloading point
### calculate road distance using API for every 20th record
#### Find the cumulative stoppage time and number of stoppage events in each trip
#### Find the min/max/median/average stoppage time for each trip (stoppage is over hours in each stoppage event)
#### Find the city name, road name and road type using the unloading point Lat/Long for each trip file


In [862]:

df_HR38 = pd.read_excel("HR38X.xltm")

In [863]:
df_HR38.head(4)

,Latitude,Longitude,Packet Time
0,13.007143,80.006966,29-Sep-2021 01:06
1,13.007143,80.006966,29-Sep-2021 01:11
2,13.007143,80.006966,29-Sep-2021 01:16
3,13.007143,80.006966,29-Sep-2021 01:21


In [864]:
df_HR38.shape

(1666, 3)

In [55]:
print(df_HR38[df_HR38["Packet Time"].isnull()][null_columns])

      Latitude  Longitude Packet Time
1665       NaN        NaN         NaN


In [865]:
df_HR38 = df_HR38[df_HR38['Packet Time'].notna()]

In [866]:
df_HR38.shape

(1665, 3)

In [867]:
df_HR38.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1665 entries, 0 to 1664
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Latitude     1665 non-null   float64
 1   Longitude    1665 non-null   float64
 2   Packet Time  1665 non-null   object 
dtypes: float64(2), object(1)
memory usage: 52.0+ KB


In [869]:
df_HR38['Packet Time'] = pd.to_datetime(df_HR38['Packet Time'])

In [39]:
#df_HR38['Packet Time_New'] = df_HR38['Packet Time'].apply(lambda x: pd.Timestamp(x).strftime('%I:%M'))

In [870]:
df_HR38['Time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df_HR38['Packet Time'].rolling(2) if len(x)>0]

In [872]:
df_HR38.Time_diff.value_counts()

0 days 00:05:00    1445
0 days 00:06:00     214
0 days 00:07:00       3
0 days 00:39:00       1
0 days 00:04:00       1
0 days 00:00:00       1
Name: Time_diff, dtype: int64

In [358]:
df_HR38['dealer_lat_hr38'] = 30.3255646
df_HR38['dealer_lon3_hr38'] = 78.0436813

In [359]:
df_HR38

,Latitude,Longitude,Packet Time,time_diff,dealer_lat_hr38,dealer_lon3_hr38,Distance
0,13.007143,80.006966,2021-09-29 01:06:00,0 days 00:00:00,30.325565,78.043681,1936.26
1,13.007143,80.006966,2021-09-29 01:11:00,0 days 00:05:00,30.325565,78.043681,1936.26
2,13.007143,80.006966,2021-09-29 01:16:00,0 days 00:05:00,30.325565,78.043681,1936.26
3,13.007143,80.006966,2021-09-29 01:21:00,0 days 00:05:00,30.325565,78.043681,1936.26
4,13.007143,80.006966,2021-09-29 01:26:00,0 days 00:05:00,30.325565,78.043681,1936.26
...,...,...,...,...,...,...,...
1660,24.524893,73.680405,2021-10-04 23:39:00,0 days 00:05:00,30.325565,78.043681,775.36
1661,24.524893,73.680405,2021-10-04 23:45:00,0 days 00:06:00,30.325565,78.043681,775.36
1662,24.524893,73.680405,2021-10-04 23:50:00,0 days 00:05:00,30.325565,78.043681,775.36
1663,24.524893,73.680405,2021-10-04 23:55:00,0 days 00:05:00,30.325565,78.043681,775.36


In [136]:
# Unloading point

In [360]:
dealer_lat_hr38 = 30.3255646
dealer_lon3_hr38 = 78.0436813
distances_km = []
for row in df_HR38.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat_hr38, dealer_lon3_hr38, row.Latitude, row.Longitude)
   )

In [361]:
df_HR38['Distance'] = distances_km

In [362]:
df_HR38.Distance.min()

648.3

In [364]:
df_HR38[0:1]

,Latitude,Longitude,Packet Time,time_diff,dealer_lat_hr38,dealer_lon3_hr38,Distance
0,13.007143,80.006966,2021-09-29 01:06:00,0 days,30.325565,78.043681,1936.26


In [363]:
df_HR38.iloc[np.where(df_HR38['Distance']==df_HR38['Distance'].min())]

,Latitude,Longitude,Packet Time,time_diff,dealer_lat_hr38,dealer_lon3_hr38,Distance
1478,25.342077,74.619530,2021-10-04 08:09:00,0 days 00:05:00,30.325565,78.043681,648.3
1479,25.342077,74.619568,2021-10-04 08:14:00,0 days 00:05:00,30.325565,78.043681,648.3
1480,25.342077,74.619568,2021-10-04 08:20:00,0 days 00:06:00,30.325565,78.043681,648.3
1481,25.342077,74.619568,2021-10-04 08:25:00,0 days 00:05:00,30.325565,78.043681,648.3
1482,25.342064,74.619560,2021-10-04 08:30:00,0 days 00:05:00,30.325565,78.043681,648.3
1483,25.342064,74.619560,2021-10-04 08:35:00,0 days 00:05:00,30.325565,78.043681,648.3
1484,25.342064,74.619560,2021-10-04 08:40:00,0 days 00:05:00,30.325565,78.043681,648.3
1485,25.342064,74.619560,2021-10-04 08:45:00,0 days 00:05:00,30.325565,78.043681,648.3
1486,25.342064,74.619560,2021-10-04 08:50:00,0 days 00:05:00,30.325565,78.043681,648.3
1487,25.342064,74.619560,2021-10-04 08:56:00,0 days 00:06:00,30.325565,78.043681,648.3


In [367]:
print(haversine((25.342077,74.619530),(30.325565,78.043681), unit='km'))

648.3035174328513


In [887]:
# stoppage point

In [124]:
df_gj1['time_diff'].value_countsnts()

0 days 00:05:00    1446
0 days 00:00:00       2
0 days 01:10:00       1
0 days 00:02:00       1
Name: time_diff, dtype: int64

In [337]:
##########
df_g93 = pd.read_excel("D:\\Sales Logistic Project\\GJ12BV6093.xltm")

In [338]:
df_g93

,"Lat,Lngt",Period
0,"13.004329,79.990362",01/10 10:10 : 01/10 10:15
1,"13.004329,79.990362",01/10 10:15 : 01/10 10:20
2,"13.004329,79.990362",01/10 10:20 : 01/10 10:25
3,"13.004329,79.990362",01/10 10:25 : 01/10 10:30
4,"13.004329,79.990362",01/10 10:30 : 01/10 10:35
...,...,...
2014,"30.381981,76.790381",08/10 10:40 : 08/10 10:45
2015,"30.381981,76.790381",08/10 10:45 : 08/10 10:50
2016,"30.381981,76.790381",08/10 10:50 : 08/10 10:55
2017,"30.381981,76.790381",08/10 10:55 : 08/10 10:56


In [339]:
df_g93 = df_g93[df_g93['Lat,Lngt'].notna()]

In [340]:
df_g93.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 0 to 2017
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Lat,Lngt  2018 non-null   object
 1   Period    2018 non-null   object
dtypes: object(2)
memory usage: 47.3+ KB


In [341]:
#splitting date col 
df_g93[['Lat', 'Lngt']] = df_g93['Lat,Lngt'].str.split(',', n=1, expand=True)

In [342]:
df_g93[0::4]

,"Lat,Lngt",Period,Lat,Lngt
0,"13.004329,79.990362",01/10 10:10 : 01/10 10:15,13.004329,79.990362
4,"13.004329,79.990362",01/10 10:30 : 01/10 10:35,13.004329,79.990362
8,"13.004329,79.990362",01/10 10:50 : 01/10 10:55,13.004329,79.990362
12,"13.004329,79.990362",01/10 11:10 : 01/10 11:15,13.004329,79.990362
16,"13.004329,79.990362",01/10 11:30 : 01/10 11:35,13.004329,79.990362
...,...,...,...,...
2000,"30.381981,76.790381",08/10 09:30 : 08/10 09:35,30.381981,76.790381
2004,"30.381981,76.790381",08/10 09:50 : 08/10 09:55,30.381981,76.790381
2008,"30.381981,76.790381",08/10 10:10 : 08/10 10:15,30.381981,76.790381
2012,"30.381981,76.790381",08/10 10:30 : 08/10 10:35,30.381981,76.790381


In [230]:
#df_g93['time_var'] = df_g93['Period'].apply(lambda x: pd.to_datetime("2021" + str(x[3:5]) + str(x[0:2]) + "-" + str(x[6:11])).strftime('%I:%M'))

In [343]:
df_g93['Time_duration'] = df_g93['Period'].map(lambda x:str(x)[6:11]) + "-" +df_g93['Period'].map(lambda x: str(x)[20:25])
df_g93['Time_duration'].head(2)

0    10:10-10:15
1    10:15-10:20
Name: Time_duration, dtype: object

In [344]:
#splitting date col 
df_g93[['St_time', 'ed_time']] = df_g93['Time_duration'].str.split('-', n=1, expand=True)

In [219]:
#df_g93['time_diff'] = [x.iloc[-1] - x.iloc[0] for x in df_g93['time_var'].rolling(2) if len(x)>0]

In [320]:
#df_g93.time_diff.value_counts()

In [321]:
#df_g93.iloc[np.where(df_g93['time_diff']==df_g93['time_diff'].min())]

In [240]:
df_g93['Time_duration'] = df_g93['Period'].map(lambda x: str(x)[6:11]) + "-" +df_g93['Period'].map(lambda x: str(x)[20:25])
df_g93['Time_duration'].head(2)

0    10:10-10:15
1    10:15-10:20
Name: Time_duration, dtype: object

In [336]:
df_g93

,"Lat,Lngt",Period,Lat,Lngt,Time_duration,St_time,ed_time,diff
0,"13.004329,79.990362",01/10 10:10 : 01/10 10:15,13.004329,79.990362,10:10-10:15,1900-01-01 10:10:00,1900-01-01 10:15:00,0 days 00:05:00
1,"13.004329,79.990362",01/10 10:15 : 01/10 10:20,13.004329,79.990362,10:15-10:20,1900-01-01 10:15:00,1900-01-01 10:20:00,0 days 00:05:00
2,"13.004329,79.990362",01/10 10:20 : 01/10 10:25,13.004329,79.990362,10:20-10:25,1900-01-01 10:20:00,1900-01-01 10:25:00,0 days 00:05:00
3,"13.004329,79.990362",01/10 10:25 : 01/10 10:30,13.004329,79.990362,10:25-10:30,1900-01-01 10:25:00,1900-01-01 10:30:00,0 days 00:05:00
4,"13.004329,79.990362",01/10 10:30 : 01/10 10:35,13.004329,79.990362,10:30-10:35,1900-01-01 10:30:00,1900-01-01 10:35:00,0 days 00:05:00
...,...,...,...,...,...,...,...,...
2013,"30.381981,76.790381",08/10 10:35 : 08/10 10:40,30.381981,76.790381,10:35-10:40,1900-01-01 10:35:00,1900-01-01 10:40:00,0 days 00:05:00
2014,"30.381981,76.790381",08/10 10:40 : 08/10 10:45,30.381981,76.790381,10:40-10:45,1900-01-01 10:40:00,1900-01-01 10:45:00,0 days 00:05:00
2015,"30.381981,76.790381",08/10 10:45 : 08/10 10:50,30.381981,76.790381,10:45-10:50,1900-01-01 10:45:00,1900-01-01 10:50:00,0 days 00:05:00
2016,"30.381981,76.790381",08/10 10:50 : 08/10 10:55,30.381981,76.790381,10:50-10:55,1900-01-01 10:50:00,1900-01-01 10:55:00,0 days 00:05:00


In [328]:
#splitting date col 
df_g93[['St_time', 'ed_time']] = df_g93['Time_duration'].str.split('-', n=1, expand=True)

In [322]:
#df['Status'] = np.where(df['Distance']!= df['Distance'].min(), 'In-transit', 'Completed')
#df_g93.iloc[np.where(df_g93['Time_duration']==df_g93['Time_duration']='00:00','24:00']

#df_g93['Time_duration'].map({'00:00': '24:00'})

In [331]:
df_g93['St_time'] = pd.to_datetime(df_g93['St_time'],format='%H:%M')

In [332]:

df_g93['ed_time'] = pd.to_datetime(df_g93['ed_time'],format='%H:%M')

In [333]:
df_g93['diff'] = df_g93['ed_time'] - df_g93['St_time']

In [334]:
df_g93['diff'].value_counts()

0 days 00:05:00      2010
-1 days +00:05:00       7
0 days 00:01:00         1
Name: diff, dtype: int64